In [1]:
%pylab inline
import tensorflow as tf
import struct, gzip
import Image
from tensorflow.examples.tutorials.mnist import input_data
from pprint import pprint

Populating the interactive namespace from numpy and matplotlib


In [2]:
data = genfromtxt('../data/exem/machine-error01.csv', skip_header=True, delimiter=',')
errors = data[:, 0]
data = data[:,2:]

train_errors = errors[:11000]
train_data = data[:11000, :]

test_errors = errors[11000:]
test_data = data[11000:, :]

In [47]:
# Parameters
learning_rate = 0.001
training_epochs = 20
batch_size = 100
display_step = 1
n_steps = 6

# Network Parameters
n_hidden_1 = 1024 # 1st layer number of features
n_hidden_2 = 512 # 2nd layer number of features
n_input = 30 # MNIST data input (img shape: 28*28)
n_classes = 10 # MNIST total classes (0-9 digits)

# tf Graph input
x = tf.placeholder("float", [None, n_input], name='x')
y = tf.placeholder("float", [2], name='y')
print y.get_shape()

(2,)


In [48]:
def next_batch(data, valid_data, n_sample=360, n_steps=n_steps):
    nrow, ncol = data.shape
    maximum = nrow - n_sample - n_steps
    idx = randint(0, maximum)
    batch_xs = data[idx:idx+n_sample,:]
    batch_y = valid_data[idx + n_sample + n_steps:idx + n_sample + n_steps + 1][0]
    batch_y = [0, 1] if batch_y == 1 else [1, 0]
    return batch_xs, batch_y


batch_xs, batch_y = next_batch(train_data, train_errors)
print batch_y, tf.shape(batch_y).get_shape(), y.get_shape()

[0, 1] (1,) (2,)


In [49]:
weight_1 = tf.Variable(tf.random_normal([n_input, n_hidden_1]), name='weight1')
bias_1 = tf.Variable(tf.random_normal([n_hidden_1]), name='bias1')
layer_1 = tf.add(tf.matmul(x, weight_1), bias_1, name='layer1')
layer_1 = tf.nn.sigmoid(layer_1)

weight_2 = tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2]), name='weight2')
bias_2 = tf.Variable(tf.random_normal([n_hidden_2]), name='bias2')
layer_2 = tf.add(tf.matmul(layer_1, weight_2), bias_2, name='layer2')
layer_2 = tf.nn.sigmoid(layer_2)

weight_3 = tf.Variable(tf.random_normal([n_hidden_2, 2]))
bias_3 = tf.Variable(tf.random_normal([2]))
layer_3 = tf.add(tf.matmul(layer_2, weight_3), bias_3)
layer_3 = tf.nn.sigmoid(layer_3)

layer_4 = tf.nn.relu(tf.reduce_sum(layer_3, reduction_indices=0))


# y_ = tf.placeholder(tf.float32, [None, 2])
# cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[0]))
# train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

# print x.get_shape()
# print weight_1.get_shape()
# print layer_1.get_shape()
# print layer_3.get_shape()
# print layer_4.get_shape()
cost = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(layer_4, y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cost)

In [50]:
# Initializing the variables
init = tf.initialize_all_variables()

# Launch the graph`
with tf.Session() as sess:
    sess.run(init)
    for i in range(10):
        for i in range(1000):
            batch_x, batch_y = next_batch(train_data, train_errors)
            _o, _c = sess.run([optimizer, cost], feed_dict={x: batch_x, y: batch_y})
        print  'cost value:', _c
    
    result = sess.run([layer_4], feed_dict={x: batch_x, y: batch_y })
    print result, batch_y
    
    # Test model
    correct_prediction = tf.equal(tf.argmax(layer_4, 0), tf.argmax(y, 0))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    
    total_accuracy = 0
    for i in range(50):
        for i in range(1000):
            batch_x, batch_y = next_batch(test_data, test_errors)
            total_accuracy += accuracy.eval({x: batch_x, y: batch_y})
    
    print 'Total Accuracy:', total_accuracy/50000.


cost value: 0.754472
cost value: 0.693087
cost value: 0.69373
cost value: 0.694117
cost value: 0.692338
cost value: 0.691239
cost value: 0.692768
cost value: 0.67142
cost value: 0.853824
cost value: 0.493681
[array([  1.14170063e+00,   8.43278322e-05], dtype=float32)] [1, 0]
Total Accuracy: 0.86746
